In [18]:
from rupo.api import Engine
engine = Engine(language="ru")

In [3]:
! pip install markovify

You are using pip version 8.1.2, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [19]:
import markovify
# Get raw text as string.
with open("corpus.txt",encoding='utf-8') as f:
    text = f.read()

# Build the model.
text_model = markovify.Text(text)

# Print five randomly-generated sentences
for i in range(5):
    print(text_model.make_sentence())

# Print three randomly-generated sentences of no more than 140 characters
for i in range(3):
    print(text_model.make_short_sentence(10))

И песня, и стих это бомба и знамя, и голос подают, - Как будто в жены их готовим скоморохам.
И то же солнце ходит надо мной, Но и я у ваших ног положить букеты и венок.
это друг мой милый.
И дальше, к деревне, быт без движеньица живут, как и в жизни был мастак.
Я раньше вас почти не чувствуя печали.
None
None
None


In [20]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

with open('corpus.txt', 'r', encoding='utf-8') as f:
    text=f.read()
vocab = sorted(set(text))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

In [21]:
def get_batches(arr, n_seqs, n_steps):
    
    characters_per_batch = n_seqs * n_steps
    n_batches = len(arr)//characters_per_batch
    
    arr = arr[:n_batches * characters_per_batch]
    
    arr = arr.reshape((n_seqs, -1))
    
    for n in range(0, arr.shape[1], n_steps):

        x = arr[:, n:n+n_steps]
        y = np.zeros_like(x)
        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        yield x, y

In [22]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)
print('x\n', x[:5, :5])
print('\ny\n', y[:5, :5])

x
 [[115 142 143 131 129]
 [160  29   1 113 143]
 [  2 133 142 134 138]
 [145 137 133 148 141]
 [143 131 157   2 144]]

y
 [[142 143 131 129   2]
 [ 29   1 113 143 142]
 [133 142 134 138  11]
 [137 133 148 141 129]
 [131 157   2 144 140]]


In [23]:
def build_inputs(batch_size, num_steps):
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

In [24]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    
    def build_cell(lstm_size, keep_prob):
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    

    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

In [25]:
def build_output(lstm_output, in_size, out_size):
    seq_output = tf.concat(lstm_output, axis=1)
    x = tf.reshape(seq_output, [-1, in_size])
    
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    logits = tf.matmul(x, softmax_w) + softmax_b
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits

In [26]:
def build_loss(logits, targets, lstm_size, num_classes):
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    return loss

In [27]:
def build_optimizer(loss, learning_rate, grad_clip):
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

In [28]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

In [29]:
batch_size = 100        # Размер пакета
num_steps = 100         # Шагов в пакете
lstm_size = 512       # Количество LSTM юнитов в скрытом слое
num_layers = 2          # Количество LSTM слоев
learning_rate = 0.001   # Скорость обучения
keep_prob = 0.5         # Dropout keep probability

In [37]:
from tqdm import tqdm
epochs = 10
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    counter = 0
    for e in tqdm(range(epochs)):
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            end = time.time()
            print('Epoch: {}/{}... '.format(e+1, epochs),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
            print("checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1/10...  Training Step: 1...  Training loss: 5.1306...  1.0671 sec/batch
checkpoints/i1_l512.ckpt
Epoch: 1/10...  Training Step: 2...  Training loss: 5.0567...  0.9667 sec/batch
checkpoints/i2_l512.ckpt
Epoch: 1/10...  Training Step: 3...  Training loss: 4.5322...  0.8716 sec/batch
checkpoints/i3_l512.ckpt
Epoch: 1/10...  Training Step: 4...  Training loss: 6.1852...  0.8556 sec/batch
checkpoints/i4_l512.ckpt
Epoch: 1/10...  Training Step: 5...  Training loss: 4.7133...  0.8688 sec/batch
checkpoints/i5_l512.ckpt
Epoch: 1/10...  Training Step: 6...  Training loss: 4.3152...  0.8601 sec/batch
checkpoints/i6_l512.ckpt
Epoch: 1/10...  Training Step: 7...  Training loss: 4.2149...  0.8539 sec/batch
checkpoints/i7_l512.ckpt
Epoch: 1/10...  Training Step: 8...  Training loss: 4.0756...  0.8680 sec/batch
checkpoints/i8_l512.ckpt
Epoch: 1/10...  Training Step: 9...  Training loss: 3.9677...  0.8657 sec/batch
checkpoints/i9_l512.ckpt
Epoch: 1/10...  Training Step: 10...  Training loss: 3.

 10%|█         | 1/10 [01:17<11:34, 77.13s/it]

Epoch: 1/10...  Training Step: 89...  Training loss: 3.4734...  0.8615 sec/batch
checkpoints/i89_l512.ckpt
Epoch: 2/10...  Training Step: 90...  Training loss: 3.5432...  0.8509 sec/batch
checkpoints/i90_l512.ckpt
Epoch: 2/10...  Training Step: 91...  Training loss: 3.4682...  0.8782 sec/batch
checkpoints/i91_l512.ckpt
Epoch: 2/10...  Training Step: 92...  Training loss: 3.4383...  0.8913 sec/batch
checkpoints/i92_l512.ckpt
Epoch: 2/10...  Training Step: 93...  Training loss: 3.4562...  0.8585 sec/batch
checkpoints/i93_l512.ckpt
Epoch: 2/10...  Training Step: 94...  Training loss: 3.4665...  0.8527 sec/batch
checkpoints/i94_l512.ckpt
Epoch: 2/10...  Training Step: 95...  Training loss: 3.4526...  0.8588 sec/batch
checkpoints/i95_l512.ckpt
Epoch: 2/10...  Training Step: 96...  Training loss: 3.4667...  0.8496 sec/batch
checkpoints/i96_l512.ckpt
Epoch: 2/10...  Training Step: 97...  Training loss: 3.4411...  0.8651 sec/batch
checkpoints/i97_l512.ckpt
Epoch: 2/10...  Training Step: 98... 

 20%|██        | 2/10 [02:34<10:16, 77.05s/it]

...

Epoch: 8/10...  Training Step: 712...  Training loss: 2.4660...  0.8657 sec/batch
checkpoints/i712_l512.ckpt
Epoch: 9/10...  Training Step: 713...  Training loss: 2.5317...  0.8581 sec/batch
checkpoints/i713_l512.ckpt
Epoch: 9/10...  Training Step: 714...  Training loss: 2.4589...  0.8350 sec/batch
checkpoints/i714_l512.ckpt
Epoch: 9/10...  Training Step: 715...  Training loss: 2.4713...  0.8583 sec/batch
checkpoints/i715_l512.ckpt
Epoch: 9/10...  Training Step: 716...  Training loss: 2.4775...  0.8545 sec/batch
checkpoints/i716_l512.ckpt
Epoch: 9/10...  Training Step: 717...  Training loss: 2.4621...  0.8531 sec/batch
checkpoints/i717_l512.ckpt
Epoch: 9/10...  Training Step: 718...  Training loss: 2.4691...  0.8998 sec/batch
checkpoints/i718_l512.ckpt
Epoch: 9/10...  Training Step: 719...  Training loss: 2.4746...  0.8539 sec/batch
checkpoints/i719_l512.ckpt
Epoch: 9/10...  Training Step: 720...  Training loss: 2.4563...  0.8634 sec/batch
checkpoints/i720_l512.ckpt
Epoch: 9/10...  Tra

 90%|█████████ | 9/10 [11:32<01:16, 76.96s/it]

Epoch: 9/10...  Training Step: 801...  Training loss: 2.4285...  0.9020 sec/batch
checkpoints/i801_l512.ckpt
Epoch: 10/10...  Training Step: 802...  Training loss: 2.4999...  0.8526 sec/batch
checkpoints/i802_l512.ckpt
Epoch: 10/10...  Training Step: 803...  Training loss: 2.4356...  0.8579 sec/batch
checkpoints/i803_l512.ckpt
Epoch: 10/10...  Training Step: 804...  Training loss: 2.4329...  0.8731 sec/batch
checkpoints/i804_l512.ckpt
Epoch: 10/10...  Training Step: 805...  Training loss: 2.4364...  0.8574 sec/batch
checkpoints/i805_l512.ckpt
Epoch: 10/10...  Training Step: 806...  Training loss: 2.4256...  0.8595 sec/batch
checkpoints/i806_l512.ckpt
Epoch: 10/10...  Training Step: 807...  Training loss: 2.4342...  0.8667 sec/batch
checkpoints/i807_l512.ckpt
Epoch: 10/10...  Training Step: 808...  Training loss: 2.4470...  0.8839 sec/batch
checkpoints/i808_l512.ckpt
Epoch: 10/10...  Training Step: 809...  Training loss: 2.4173...  0.8755 sec/batch
checkpoints/i809_l512.ckpt
Epoch: 10/1

100%|██████████| 10/10 [12:49<00:00, 76.93s/it]

Epoch: 10/10...  Training Step: 890...  Training loss: 2.3879...  0.8618 sec/batch
checkpoints/i890_l512.ckpt


In [38]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [42]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="Сегодня 17 октября 2018 года"):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [43]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i890_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i890_l512.ckpt"

In [45]:
samp = sample('checkpoints/i890_l512.ckpt',100, lstm_size, len(vocab))
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i890_l512.ckpt
Сегодня 17 октября 2018 года в дем 
протет поледани, стаховить в нем, 
потерет, 
поднетье в дули, кок небель 
нам на стратали нет
